In [1]:
import numpy as np
import random
import torch
import argparse
import sys

from utils import seed_everything
from dataset import get_dataset
from community_detection import hierarchical_structure_generation
from execution import execute_NC, execute_LP
from preparation import LP_preparation, NC_preparation

import warnings
warnings.filterwarnings('ignore')

In [2]:
sys.argv = ['']  # execution on jupyter notebook
parser = argparse.ArgumentParser()
# general
parser.add_argument('--task', dest='task', default='LP', type=str,
                    help='LP; NC; Inductive')
parser.add_argument('--mode', dest='mode', default='baseline', type=str,
                    help='experiment mode. E.g., baseline or basemodel')
parser.add_argument('--model', dest='model', default='GCN', type=str,
                    help='model class name. E.g., GCN, PGNN, HCGNN...')
parser.add_argument('--dataset', dest='dataset', default='grid', type=str,
                    help='cora; grid; communities; ppi')
parser.add_argument('--gpu', dest='gpu', default=True, type=bool,
                    help='whether use gpu')
parser.add_argument('--SEED', dest='SEED', default=123, type=int)

# dataset
parser.add_argument('--ratio_sample_pos_link', dest='ratio_sample_pos_link', default=20, type=float)
parser.add_argument('--use_features', dest='use_features', default=True, type=bool,
                    help='whether use node features')
parser.add_argument('--community_detection_method', dest='community_detection_method', default='Louvain', type=str,
                    help='community detection method, default Louvain')
parser.add_argument('--threshold', dest='threshold', default=1, type=int,
                    help='the threshold for graph generation, default 1')

# model
parser.add_argument('--lr', dest='lr', default=1e-2, type=float)
parser.add_argument('--epoch_num', dest='epoch_num', default=201, type=int)
parser.add_argument('--epoch_log', dest='epoch_log', default=10, type=int)
parser.add_argument('--layer_num', dest='layer_num', default=2, type=int)
parser.add_argument('--relu', dest='relu', default=True, type=bool)
parser.add_argument('--dropout', dest='dropout', default=False, type=bool)
parser.add_argument('--drop_ratio', dest='drop_ratio', default=0.5, type=float)
parser.add_argument('--feature_pre', dest='feature_pre', default=True, type=bool)
parser.add_argument('--same_level_gnn', dest='same_level_gnn', default='GCN', type=str,
                    help='agg within level. E.g., MEAN GCN, SAGE, GAT, GIN, ...')
parser.add_argument('--down2up_gnn', dest='down2up_gnn', default='MEAN', type=str,
                    help='aggregation bottom-up. E.g., MEAN, GCN, SAGE, GAT, GIN, ...')
parser.add_argument('--up2down_gnn', dest='up2down_gnn', default='GAT', type=str,
                    help='aggregation top-down. E.g., GCN, SAGE, GAT, GIN, ...')
parser.add_argument('--fshot', dest='fshot', default=False, type=bool)

parser.set_defaults(gpu=False, task='LP', model='GCN', dataset='grid', feature_pre=True)
args = parser.parse_args()

In [3]:
args.task = 'NC'
args.dataset = 'cora'
args.mode = 'basemodel'
args.model = 'HCGNN'
args.layer_num = 2
args.epoch_num = 201
args.lr = 0.01
args.relu = True
args.dropout = False
args.drop_ratio = 0.5
args.same_level_gnn = 'GCN'
args.down2up_gnn = 'MEAN'
args.up2down_gnn = 'GCN'
args.fshot = True
args.SEED = 1234
args.gpu = True
args.device = torch.device('cuda:'+str(0) if args.gpu and torch.cuda.is_available() else 'cpu')

seed_everything(args.SEED)

print(args, '\n')

Namespace(SEED=1234, community_detection_method='Louvain', dataset='cora', device=device(type='cuda', index=0), down2up_gnn='MEAN', drop_ratio=0.5, dropout=False, epoch_log=10, epoch_num=201, feature_pre=True, fshot=True, gpu=True, layer_num=2, lr=0.01, mode='basemodel', model='HCGNN', ratio_sample_pos_link=20, relu=True, same_level_gnn='GCN', task='NC', threshold=1, up2down_gnn='GCN', use_features=True) 



In [4]:
graphs, features, df_labels = get_dataset(
    dataset_name=args.dataset,
    use_features=args.use_features,
    task=args.task,
)
ls_hierarchical_community, ls_up2down_edges, ls_down2up_edges = hierarchical_structure_generation(
    dataset_name=args.dataset,
    graphs=graphs,
    method=args.community_detection_method,
    threshold=args.threshold
)
ls_adj_same_level = NC_preparation(
    graphs=graphs,
    ls_hierarchical_community=ls_hierarchical_community
)
execute_NC(
    args, graphs, df_labels, features, ls_hierarchical_community,
    ls_adj_same_level, ls_up2down_edges, ls_down2up_edges,
)

is reading cora dataset...
Name: 
Type: Graph
Number of nodes: 2708
Number of edges: 5278
Average degree:   3.8981
is processing dataset...
is generating hierarchical structure....
Is doing community detection....
start 0 subgraph...
iteration 0: from 2708 items to 708 items, modularity is 0.5677445259918352.
iteration 1: from 708 items to 197 items, modularity is 0.7765386604092458.
iteration 2: from 197 items to 111 items, modularity is 0.8123942736126004.
iteration 3: from 111 items to 104 items, modularity is 0.8141100187656457.
Community detection is done
is presenting hierarchical structure....
start graph community 0
layer 0: keys: [2708, 3415], values: [0, 2707]
layer 1: keys: [3416, 3612], values: [2708, 3415]
layer 2: keys: [3613, 3723], values: [3416, 3612]
layer 3: keys: [3724, 3827], values: [3613, 3723]
Is setting up hierarchical pipelines....
start graph 0
Hierarchical pipelines are ready
is recording up2down edges....
keys in [2708, 3827], values in [0, 2707]
is recordi

Best Valid Mac-f1 is 0.7680, best Test Mac-f1 is 0.7690 and epoch_id is 40
Epoch 160 starts !
--------------------------------------------------------------------------------
Evaluating Epoch 160, time 0.136
Micro-f1:  Train = 1.0000, Valid = 0.7660, Test Micro-f1 = 0.7530
Macro-f1: Train = 1.0000, valid = 0.7580, Macro-f1 = 0.7588
Best Valid Mic-f1 is 0.7780, best Test Mic-f1 is 0.7670 and epoch_id is 40
Best Valid Mac-f1 is 0.7680, best Test Mac-f1 is 0.7690 and epoch_id is 40
Epoch 170 starts !
--------------------------------------------------------------------------------
Evaluating Epoch 170, time 0.128
Micro-f1:  Train = 1.0000, Valid = 0.7420, Test Micro-f1 = 0.7440
Macro-f1: Train = 1.0000, valid = 0.7321, Macro-f1 = 0.7484
Best Valid Mic-f1 is 0.7780, best Test Mic-f1 is 0.7670 and epoch_id is 40
Best Valid Mac-f1 is 0.7680, best Test Mac-f1 is 0.7690 and epoch_id is 40
Epoch 180 starts !
--------------------------------------------------------------------------------
Evaluat